# Chap 1. 신경망 복습

##### 1.1. 수학과 파이썬 복습은 패스하겠습니다

## 1.2. 신경망의 추론


### 신경망
- 입력을 출력으로 변환하는 단순한 함수
- 입력층, 은닉층, 출력층으로 구성
- 가중치를 학습시켜 원하는 형태 구성 및 이전 뉴런의 값에 영향을 받지 않는 편향 존재

![](img/chap_01/1.png)

- 은닉층의 첫번째 뉴런의 계산 공식
    - ![](img/chap_01/2.png)

- 실제 완전연결 계층이 수행하는 변환을 다음과 같이 행렬의 곱으로 정리 가능
    - ![](img/chap_01/3.png)
    - 다음과 같이 간소화 할 수 있음
        - ![](img/chap_01/4.png)
        
- 샘플 데이터에 따른 데이터 형상 확인

![](img/chap_01/5.png)
![](img/chap_01/6.png)

### 선형 변환과 비선형 변환
- 일반적인 완전연결계층에 의한 변환은 선형 변환임
- 선형 변환만으로는 은닉층이 없는 네트워크와 다를 바가 없음
- 비선형 활성화 함수를 이용하여 신경망의 표현력을 높일 수 있음

#### Sigmoid function
- 기본적인 활성화 함수
- 임의의 실수를 입력받아 0 ~ 1 사이의 실수를 출력
- 선형 회귀에서 많이 사용

![](img/chap_01/7.png)

![](img/chap_01/8.png)

### Affine 계층과 Sigmoid 계층의 순전파
- 각 계층을 파이썬 클래스로 구현
- 모든 계층은 forward()와 backward()메서드를 가짐
- 모든 계층은 params와 grads를 인스턴스 변수로 가짐
- params: 가중치 및 편향 등의 값을 담는 리스트
- grads: params에 저장된 매개변수에 대응하는 기울기를 담는 리스트

## 1.3. 신경망의 학습


### Loss function
- 신경망의 학습이 얼마나 잘 되고 있는가를 알기 위한 척도 필요
- 손실을 사용하여 학습 단계의 특정 시점의 신경망의 성능을 나타냄
- 손실은 정답 라벨과 신경망이 예측한 결과를 비교하여 예측의 차이를 나타내는 스칼라 값

### Softmax
- n개의 입력에 대해 n개의 출력을 내보내며, 각 출력은 0.0 이상 1.0 이하의 실수, 모든 출력의 합은 1.0 => 각 출력을 확률로 해석 가능
- 출력이 총 n개일 때, k번째 출력 y_k를 구하는 계산식
    
    ![](img/chap_01/10.png)

### Cross Entropy Error
- 다중 클래스 분류 신경망에서 흔히 사용하는 손실 함수
- 신경망이 출력하는 각 클래스의 확률과 정답 레이블을 이용해 구할 수 있음
- 출력이 총 k개일 때, cross entropy error를 구하는 계산식

    ![](img/chap_01/11.png)
    - t_k는 k번째 클래스에 해당하는 정답 레이블(0 or 1)
    - 정답 레이블은 t = [0, 0, 1]과 같이 ont-hot vector로 표기
    - y_k는 k번째 출력의 softmax를 계산한 확률값
    
- 실제로 cross entropy error은 정답 레이블이 1인 원소의 출력값의 자연로그 값
- 다른 원소들은 t_k(정답레이블)가 0이기 때문에 결과에 영향을 주지 못함

#### 미니 배치를 고려한 Cross Entropy Error
- 데이터가 N개이며, t_nk는 n번째 데이터의 k차원째의 값(0 or 1)
- 데이터를 N개로 확장하였고, 이를 N으로 나눠 1개당 평균 손실함수를 구함
    
    ![](img/chap_01/12.png)

![](img/chap_01/9.png)
       
#### 일반적으로 softmax와 cross entropy를 Softmax with Loss 계층 하나로 구현하며, 이로인해 역전파 계산이 쉬워짐
![](img/chap_01/13.png) 

### 미분과 기울기
- 벡터의 각 원소에 대한 미분을 정리한 것이 기울기

    ![](img/chap_01/15.png) 

- 벡터와 마찬가지로 행렬에서도 기울기를 정의할 수 있음
    - W가 m x n 행렬이라면, L = g(W) 함수의 기울기는 다음과 같음
    - 이때, W와 $\frac{\partial L}{\partial W}$의 형상이 같음
    
        ![](img/chap_01/14.png) 
        
        
### 오차역전파
- 학습 시 신경망은 학습 데이터를 입력하면 손실을 출력
- 각 매개변수에 대한 손실의 기울기를 구하고 싶음
- 오차역전파를 사용하여 손쉽게 기울기를 구할 수 있음

### 연쇄법칙
- 합성함수에 대한 미분 법칙
- 아무리 많은 함수를 연결해도, 각 함수의 국소적인 미분을 계산할 수 있다면 그 값들을 곱해서 전체의 미분을 구할 수 있음

### 각 노드별 순전파 및 역전파 개념 및 그래프
#### 덧셈 노드
- $\frac{\partial z}{\partial x}$와 $\frac{\partial z}{\partial y}$ 모두 1
- 덧셈 노드는 상류로부터 받은 값에 1을 곱하여 하류로 기울기를 전파 = 기울기를 그대로 전파

![](img/chap_01/16.png) 

#### 곱셈 노드
- $\frac{\partial z}{\partial x} = y$, $\frac{\partial z}{\partial y} = x$
- 곱셈 노드는 순전파 시의 입력을 서로 바꾼 값을 곱함

![](img/chap_01/17.png) 

#### 분기 노드
- 같은 값이 복제되어 분기되었기 때문에 역전파는 상류에서 온 기울기들의 합

![](img/chap_01/18.png)

#### Repeat 노드
- 분기노드를 일반화한 노드

![](img/chap_01/19.png) 

#### Sum 노드
- 범용 덧셈 노드로, $N \times D$ 배열을 0축(axis=0)에 대한 합을 구함
- Sum 노드의 역전파는 상류로부터의 기울기를 하류로 분배 => 덧셈 노드의 확장
- Sum 노드와 Repeat 노드는 서로 반대 관계

![](img/chap_01/20.png) 

#### MATtatMul
 ㄴ노녿노드

## 1.4. 신경망으로 문제를 풀다


##### 1.5. 계산 고속화는 패스하겠습니다.

# Chap 2. 자연어와 단어의 분산 표현

## 2.1. 자연어 처리란
- 우리가 사용하는 말을 컴퓨터에게 이해시키기 위한 기술
- 컴퓨터에게 자연어를 이해시킬 수 있다면, 검색엔진, 기계번역 등과 같은 기술에 널리 사용 가능

### 단어의 의미
- 말은 문자로 구성되며, 말의 의미는 단어로 구성
- 단어란, 의미의 최소단위이며, 컴퓨터에게 단어의 의미를 이해시키는 것은 중요
- '시소러스', '통계 기반 기법', '추론 기반 기법' 등을 통해 단어의 의미를 표현할 수 있음

## 2.2. 시소러스
- 우리가 각각의 단어에 의미를 설명해 넣는 것처럼, 컴퓨터가 이해할 수 있는 형태의 사전 형식
- 뜻이 같은 단어나 뜻이 비슷한 단어가 한 그룹으로 분류

    ![](img/chap_01/21.png) 
    
- 단어 사이의 상위-하위 혹은 전체-부분 등 의 관계까지 묘사할 수 있음

    ![](img/chap_01/22.png) 
    
- 이러한 단어 네트워크를 통해 컴퓨가 간접적으로 단어의 의미를 이해했다고 할 수 있음
- 자연어 처리 분야에서 WordNet이라는 시소러스가 가장 유명함

### 시소러스의 문제점
- 시대 변화에 대응하기 어려움
- 사람을 쓰는 비용이 큼
- 단어의 미묘한 차이를 표현할 수 없음

#### => 이를 해결하기 위해 통계 기반 기법과 신경망을 사용한 추론 기반 기법을 이용

## 2.3. 통계 기반 기법
- 말뭉치라는 대량의 텍스트 데이터 사용
- 자연어 처리 연구나 애플리케이션을 염두에 두고 수집된 텍스트 데이터
- 말뭉치에는 자연어에 대한 사람의 지식이 충분히 담겨있다고 볼 수 있음
- 사람의 지식으로 가득한 말뭉치에서 자동으로, 효율적으로 그 핵심을 추출하는 것이 목표

### 파이썬으로 말뭉치 전처리하기
- 텍스트 데이터를 토큰(여기서는 단어) 단위로 분할
    - You say goodbye and I say hello. => ['You', 'say', 'goodbye', 'and', 'I', 'say', 'hello', '.']
- 단어를 텍스트 그대로 사용하는 것은 힘들기 때문에 단어에 ID를 부여
    - ['You', 'say', 'goodbye', 'and', 'I', 'say', 'hello', '.'] => {0: 'You', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'I', 5: 'hello', 6: '.'}
- 단어 목록을 단어 ID목록으로 변경하여 이후에 사용할 수 있도록 함
    - You say goodbye and I say hello. => [0, 1, 2, 3, 4, 1, 5, 6]

### 단어의 분산 표현
- 색상을 표현하는 방식은 크게 두 가지
    1. 고유한 이름을 부여하여 표(색의 가짓수만큼의 이름이 필요)
    2. RGB(Red/Green/Blue)의 세가지 성분의 비율로 표현(3차원의 벡터로 표현)
- 벡터를 사용할 경우, 관련성 파악 및 정량화가 훨씬 쉬워짐
- 색상처럼, 단어도 단어의 의미를 정확하게 파악할 수 있는 벡터 표현로 표현할 수 있음
- 이러한 단어의 분산 표현을 어떻게 구축할 것인가가 중요한 주제

### 분포 가설
- 단어 자체에는 의미가 없고, 그 단어가 사용되는 context가 의미를 형성
    - "I drink beer"와 "We drink wine"가 있다면, 'drink'라는 단어 주변에는 음료가 등장하기 쉬움
    
![](img/chap_01/23.png)

### 동시발생 행렬
- 어떤 단어에 주목했을 때, 주변에 어떤 단어가 몇 번 등장했는지 세어 집계하는 방법
- 주변 단어를 몇개 포함할지 window size를 통해 결정
- you와 say의 context 확인(window size=1)

    ![](img/chap_01/24.png) ![](img/chap_01/25.png)
    
    ![](img/chap_01/26.png) ![](img/chap_01/27.png)

- 모든 단어에 대해 context를 확인해보면 다음과 같은 동시발생 행렬이 됨

    ![](img/chap_01/28.png)
    
### 벡터 간 유사도
- 단어를 벡터로 나타낼 수 있기 때문에 벡터 연산이 가능
- 각 벡터간의 유사도를 구할 수 있음
- 일반적으로 consine similarity를 자주 사용
- 두 벡터 $x = (x_1, x_2, x_3, ..., x_n)$과 $y = (y_1, y_2, y_3, ..., y_n)$이 있을 때, cosine similarity는 다음과 같음

    ![](img/chap_01/29.png)

## 2.4. 통계 기반 기법 개선하기
- 동시발생 행렬을 통해 단어를 벡터로 표현하였지만, 개선이 필요
- 더 큰 말뭉치를 이용하여 진짜 단어의 분산 표현을 얻어볼 필요가 있음

### 상호정보량
- 단순히 두 단어가 동시에 발생한 횟수가 좋은 특징은 아님
    - 'the'와 'car'의 동시발생 횟수는 매우 많지만, 실제로는 'car'라는 단어는 'drive'와 더 관련이 깊음
    - 이는 'the'라는 단어가 고빈도 단어여서 'car'와 강한 관련성을 갖는다고 평가되기 때문

#### Pointwise Mutual Information(PMI)
- $P(x)$: 단어 x가 말뭉치에 등장할 확률
- $P(y)$: 단어 y가 말뭉치에 등장할 확률
- $P(x, y)$: 단어 x와 y가 동시 발생할 확률
- 두 단어의 PMI값이 높을수록 관련성이 높음
- $PMI(x, y) = log_2\frac{P(x, y)}{P(x)P(y)}$

- $PMI(x, y) = log_2\frac{P(x, y)}{P(x)P(y)} = log_2\frac{\frac{C(x, y)}{N}}{\frac{C(x)}{N}\frac{C(y)}{N}} = log_2\frac{C(x, y) \cdot N}{C(x)C(y)}$

- $PMI('the', 'car')$와 $PMI('car', 'drive')$
    - 말뭉치 단어 수 N = 10000
    - 각 단어 'the', 'car', 'drive'의 등장 횟수 = 1000, 20, 10
    - 'the'와 'car'의 동시발생수 = 10
    - 'car'와 'drive'의 동시발생수 = 5
    
    $PMI('the', 'car') = log_2\frac{10 \cdot 10000}{1000 \cdot 20} \approx 2.32$
    
    $PMI('car', 'drive') = log_2\frac{5 \cdot 10000}{20 \cdot 10} \approx 7.97$
    
- PMI는 단어가 단독으로 출현하는 횟수를 고려하기 때문에, 자주 등장한 "the" 때문에 $PMI('the', 'car')$는 낮은 점수를 가짐

#### Positive PMI(PPMI)
- PMI는 두 단어의 동시발생 횟수가 0이면 $log_2 0 = -\infty$가 되기 때문에 실제로는 PPMI를 사용
- PPMI는 PMI값이 음수일 경우, 0으로 취급

    $PPMI(x, y) = max(0, PMI(x, y))$
    
#### 차원 감소
- PPMI를 이용한 행렬에는 말뭉치의 어휘가 증가함에 따라 단어벡터의 차원수도 증가한다는 문제가 있음